In [10]:
import pandas as pd


df = pd.read_csv('measuring_hate_speech 2.csv')

selected_columns = df[['comment_id', 'platform', 'sentiment','violence',  'hatespeech', 'hate_speech_score', 'text', 'target_race', 'target_religion', 'target_origin', 'target_gender', 'target_sexuality', 'target_age', 'target_disability']]
print(selected_columns)


        comment_id  platform  sentiment  violence  hatespeech  \
0            47777         3        0.0       0.0         0.0   
1            39773         2        0.0       0.0         0.0   
2            47101         3        4.0       0.0         2.0   
3            43625         3        2.0       0.0         0.0   
4            12538         0        4.0       4.0         2.0   
...            ...       ...        ...       ...         ...   
135551       37080         2        1.0       0.0         0.0   
135552       22986         2        2.0       0.0         0.0   
135553       21008         2        1.0       0.0         0.0   
135554       22986         2        2.0       0.0         0.0   
135555       14785         0        4.0       3.0         2.0   

        hate_speech_score                                               text  \
0                   -3.90  Yes indeed. She sort of reminds me of the elde...   
1                   -6.52  The trans women reading this twe

In [25]:
print(selected_columns['comment_id'].max())
print(selected_columns['comment_id'].min())

50070
1


In [11]:

selected_columns = selected_columns.replace({True: 1, False: 0})
print(selected_columns)


        comment_id  platform  sentiment  violence  hatespeech  \
0            47777         3        0.0       0.0         0.0   
1            39773         2        0.0       0.0         0.0   
2            47101         3        4.0       0.0         2.0   
3            43625         3        2.0       0.0         0.0   
4            12538         0        4.0       4.0         2.0   
...            ...       ...        ...       ...         ...   
135551       37080         2        1.0       0.0         0.0   
135552       22986         2        2.0       0.0         0.0   
135553       21008         2        1.0       0.0         0.0   
135554       22986         2        2.0       0.0         0.0   
135555       14785         0        4.0       3.0         2.0   

        hate_speech_score                                               text  \
0                   -3.90  Yes indeed. She sort of reminds me of the elde...   
1                   -6.52  The trans women reading this twe

In [27]:

grouped_columns = selected_columns.groupby('comment_id')[['target_race', 'target_religion', 'target_origin', 'target_gender', 'target_sexuality', 'target_age', 'target_disability']].sum()

print(grouped_columns)

            target_race  target_religion  target_origin  target_gender  \
comment_id                                                               
1                     4                0              0              0   
2                     0                0              0              0   
3                     3                0              0              0   
4                     0                0              0              0   
5                     3                0              0              0   
...                 ...              ...            ...            ...   
50066                 1                0              3              0   
50067                 0                0              1              0   
50068                 0                0              1              1   
50069                 0                0              0              1   
50070                 0                0              0              3   

            target_sexuality  target_

In [69]:
import pandas as pd
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df = pd.read_csv('measuring_hate_speech 2.csv')

selected_columns = df[['comment_id', 'platform', 'sentiment', 'violence', 'hatespeech', 'hate_speech_score', 'text', 'target_race', 'target_religion', 'target_origin', 'target_gender', 'target_sexuality', 'target_age', 'target_disability']]


selected_columns = selected_columns.replace({True: 1, False: 0})


grouped_columns = selected_columns.groupby('comment_id')[['platform','sentiment','violence','hatespeech','hate_speech_score','target_race', 'target_religion', 'target_origin', 'target_gender', 'target_sexuality', 'target_age', 'target_disability']].sum()

selected_columns_unique = selected_columns.drop_duplicates(subset='comment_id', keep='first')


final_df = pd.merge(grouped_columns, selected_columns_unique[['comment_id', 'text']], on='comment_id', how='left')
final_df['text']=final_df['text'].apply(preprocess_text)

print(final_df)


       comment_id  platform  sentiment  violence  hatespeech  \
0               1         0       15.0       1.0         0.0   
1               2         0       10.0       2.0         4.0   
2               3         0        7.0       0.0         5.0   
3               4         0        7.0       1.0         3.0   
4               5         0        6.0       0.0         0.0   
...           ...       ...        ...       ...         ...   
39560       50066        12       16.0      11.0         6.0   
39561       50067         3        4.0       3.0         2.0   
39562       50068         6        1.0       0.0         0.0   
39563       50069         3        4.0       1.0         0.0   
39564       50070         9       12.0       8.0         4.0   

       hate_speech_score  target_race  target_religion  target_origin  \
0                   1.84            4                0              0   
1                   0.09            0                0              0   
2           

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(final_df['text'])

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import time
tfidf = TfidfVectorizer()


X_text = tfidf.fit_transform(final_df['text'])


X_train, X_test, y_train, y_test = train_test_split(X_text, final_df['target_race'], test_size=0.2, random_state=42)


learning_rates = [0.001, 0.01, 0.1, 0.5, 1]


lr_losses = {}


lr_training_times = {}


for lr in learning_rates:
    start_time = time.time()
    
    catboost_model = CatBoostClassifier(learning_rate=lr, iterations=100)  
    catboost_model.fit(X_train, y_train, verbose=False)
    end_time = time.time()
    training_time = end_time - start_time
    lr_training_times[lr] = training_time
    y_pred = catboost_model.predict(X_test)
    loss = 1 - accuracy_score(y_test, y_pred)  
    lr_losses[lr] = loss


optimal_lr = min(lr_losses, key=lr_losses.get)
min_loss = lr_losses[optimal_lr]
print("Optimal Learning Rate:", optimal_lr)
print("Minimum Loss:", min_loss)
print("Training Time with Optimal Learning Rate:", lr_training_times[optimal_lr])

start_time = time.time()
catboost_model_optimal_lr = CatBoostClassifier(learning_rate=optimal_lr, iterations=100) 
catboost_model_optimal_lr.fit(X_train, y_train, verbose=False)
end_time = time.time()
training_time_optimal_lr = end_time - start_time
print("Total Training Time with Optimal Learning Rate:", training_time_optimal_lr)

y_pred_optimal_lr = catboost_model_optimal_lr.predict(X_test)
accuracy_optimal_lr = accuracy_score(y_test, y_pred_optimal_lr)
print("Accuracy with Optimal Learning Rate:", accuracy_optimal_lr)

start_time = time.time()
catboost_model = CatBoostClassifier()
catboost_model.fit(X_train, y_train)
end_time = time.time()
training_time_default = end_time - start_time
print("Total Training Time with Default Hyperparameters:", training_time_default)

y_pred = catboost_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy with Default Hyperparameters:", accuracy)


Optimal Learning Rate: 0.1
Minimum Loss: 0.3245292556552508
Training Time with Optimal Learning Rate: 1437.452731847763
Total Training Time with Optimal Learning Rate: 251.7605721950531
Accuracy with Optimal Learning Rate: 0.6754707443447492
Learning rate set to 0.094372
0:	learn: 2.0977503	total: 2.29s	remaining: 38m 4s
1:	learn: 1.8851718	total: 4.79s	remaining: 39m 48s
2:	learn: 1.7478714	total: 7.19s	remaining: 39m 50s
3:	learn: 1.6378082	total: 10.9s	remaining: 45m 11s
4:	learn: 1.5506068	total: 13.4s	remaining: 44m 35s
5:	learn: 1.4786243	total: 16s	remaining: 44m 15s
6:	learn: 1.4201041	total: 18.5s	remaining: 43m 41s
7:	learn: 1.3691362	total: 20.9s	remaining: 43m 15s
8:	learn: 1.3264207	total: 23.4s	remaining: 42m 54s
9:	learn: 1.2853719	total: 25.9s	remaining: 42m 40s
10:	learn: 1.2501261	total: 28.3s	remaining: 42m 26s
11:	learn: 1.2195014	total: 30.8s	remaining: 42m 15s
12:	learn: 1.1911628	total: 33.1s	remaining: 41m 54s
13:	learn: 1.1660774	total: 35.6s	remaining: 41m 45s

148:	learn: 0.8490153	total: 7m 29s	remaining: 42m 48s
149:	learn: 0.8486626	total: 7m 32s	remaining: 42m 42s
150:	learn: 0.8480811	total: 7m 35s	remaining: 42m 43s
151:	learn: 0.8476271	total: 7m 39s	remaining: 42m 42s
152:	learn: 0.8471763	total: 7m 41s	remaining: 42m 36s
153:	learn: 0.8467655	total: 7m 44s	remaining: 42m 29s
154:	learn: 0.8464877	total: 7m 46s	remaining: 42m 23s
155:	learn: 0.8461413	total: 7m 49s	remaining: 42m 17s
156:	learn: 0.8458423	total: 7m 51s	remaining: 42m 11s
157:	learn: 0.8455833	total: 7m 53s	remaining: 42m 5s
158:	learn: 0.8451710	total: 7m 56s	remaining: 41m 59s
159:	learn: 0.8449088	total: 7m 59s	remaining: 41m 56s
160:	learn: 0.8445505	total: 8m 1s	remaining: 41m 51s
161:	learn: 0.8439292	total: 8m 4s	remaining: 41m 45s
162:	learn: 0.8436240	total: 8m 6s	remaining: 41m 40s
163:	learn: 0.8432159	total: 8m 10s	remaining: 41m 37s
164:	learn: 0.8426521	total: 8m 12s	remaining: 41m 32s
165:	learn: 0.8422940	total: 8m 14s	remaining: 41m 26s
166:	learn: 0.

297:	learn: 0.8114543	total: 14m 4s	remaining: 33m 9s
298:	learn: 0.8112309	total: 14m 7s	remaining: 33m 6s
299:	learn: 0.8109928	total: 14m 10s	remaining: 33m 4s
300:	learn: 0.8107740	total: 14m 12s	remaining: 33m
301:	learn: 0.8105813	total: 14m 15s	remaining: 32m 56s
302:	learn: 0.8103185	total: 14m 17s	remaining: 32m 52s
303:	learn: 0.8101957	total: 14m 20s	remaining: 32m 50s
304:	learn: 0.8100040	total: 14m 23s	remaining: 32m 47s
305:	learn: 0.8099109	total: 14m 26s	remaining: 32m 45s
306:	learn: 0.8098274	total: 14m 30s	remaining: 32m 44s
307:	learn: 0.8095692	total: 14m 33s	remaining: 32m 43s
308:	learn: 0.8094050	total: 14m 37s	remaining: 32m 42s
309:	learn: 0.8092414	total: 14m 42s	remaining: 32m 44s
310:	learn: 0.8090454	total: 14m 46s	remaining: 32m 43s
311:	learn: 0.8089283	total: 14m 49s	remaining: 32m 41s
312:	learn: 0.8086360	total: 14m 52s	remaining: 32m 39s
313:	learn: 0.8085262	total: 14m 55s	remaining: 32m 37s
314:	learn: 0.8083648	total: 14m 59s	remaining: 32m 35s
3

445:	learn: 0.7926409	total: 22m 41s	remaining: 28m 11s
446:	learn: 0.7925483	total: 22m 45s	remaining: 28m 9s
447:	learn: 0.7924151	total: 22m 49s	remaining: 28m 6s
448:	learn: 0.7923512	total: 22m 53s	remaining: 28m 5s
449:	learn: 0.7923022	total: 22m 57s	remaining: 28m 3s
450:	learn: 0.7922708	total: 23m	remaining: 28m
451:	learn: 0.7919640	total: 23m 4s	remaining: 27m 58s
452:	learn: 0.7919120	total: 23m 7s	remaining: 27m 55s
453:	learn: 0.7918502	total: 23m 11s	remaining: 27m 53s
454:	learn: 0.7916772	total: 23m 14s	remaining: 27m 50s
455:	learn: 0.7915560	total: 23m 17s	remaining: 27m 47s
456:	learn: 0.7914914	total: 23m 20s	remaining: 27m 44s
457:	learn: 0.7914076	total: 23m 24s	remaining: 27m 41s
458:	learn: 0.7913387	total: 23m 27s	remaining: 27m 38s
459:	learn: 0.7912820	total: 23m 30s	remaining: 27m 35s
460:	learn: 0.7911751	total: 23m 33s	remaining: 27m 32s
461:	learn: 0.7911296	total: 23m 36s	remaining: 27m 29s
462:	learn: 0.7910468	total: 23m 39s	remaining: 27m 26s
463:	l

593:	learn: 0.7807078	total: 31m 23s	remaining: 21m 27s
594:	learn: 0.7805475	total: 31m 27s	remaining: 21m 24s
595:	learn: 0.7805191	total: 31m 30s	remaining: 21m 21s
596:	learn: 0.7804210	total: 31m 33s	remaining: 21m 18s
597:	learn: 0.7803386	total: 31m 37s	remaining: 21m 15s
598:	learn: 0.7802516	total: 31m 40s	remaining: 21m 12s
599:	learn: 0.7802015	total: 31m 43s	remaining: 21m 8s
600:	learn: 0.7800464	total: 31m 46s	remaining: 21m 5s
601:	learn: 0.7799833	total: 31m 49s	remaining: 21m 2s
602:	learn: 0.7798753	total: 31m 52s	remaining: 20m 59s
603:	learn: 0.7798241	total: 31m 55s	remaining: 20m 56s
604:	learn: 0.7797472	total: 31m 59s	remaining: 20m 53s
605:	learn: 0.7796944	total: 32m 3s	remaining: 20m 50s
606:	learn: 0.7796515	total: 32m 9s	remaining: 20m 49s
607:	learn: 0.7796132	total: 32m 12s	remaining: 20m 45s
608:	learn: 0.7795587	total: 32m 15s	remaining: 20m 42s
609:	learn: 0.7795159	total: 32m 18s	remaining: 20m 39s
610:	learn: 0.7794850	total: 32m 21s	remaining: 20m 3

741:	learn: 0.7699537	total: 39m 23s	remaining: 13m 41s
742:	learn: 0.7699061	total: 39m 26s	remaining: 13m 38s
743:	learn: 0.7698466	total: 39m 29s	remaining: 13m 35s
744:	learn: 0.7697937	total: 39m 35s	remaining: 13m 32s
745:	learn: 0.7697419	total: 39m 39s	remaining: 13m 30s
746:	learn: 0.7696858	total: 39m 42s	remaining: 13m 26s
747:	learn: 0.7696382	total: 39m 45s	remaining: 13m 23s
748:	learn: 0.7695752	total: 39m 49s	remaining: 13m 20s
749:	learn: 0.7695306	total: 39m 52s	remaining: 13m 17s
750:	learn: 0.7694650	total: 39m 55s	remaining: 13m 14s
751:	learn: 0.7694391	total: 39m 58s	remaining: 13m 11s
752:	learn: 0.7693082	total: 40m 2s	remaining: 13m 7s
753:	learn: 0.7692853	total: 40m 5s	remaining: 13m 4s
754:	learn: 0.7692580	total: 40m 9s	remaining: 13m 1s
755:	learn: 0.7691940	total: 40m 12s	remaining: 12m 58s
756:	learn: 0.7691682	total: 40m 15s	remaining: 12m 55s
757:	learn: 0.7690854	total: 40m 18s	remaining: 12m 52s
758:	learn: 0.7689737	total: 40m 21s	remaining: 12m 48

890:	learn: 0.7618532	total: 47m 21s	remaining: 5m 47s
891:	learn: 0.7617727	total: 47m 24s	remaining: 5m 44s
892:	learn: 0.7616912	total: 47m 27s	remaining: 5m 41s
893:	learn: 0.7616518	total: 47m 30s	remaining: 5m 37s
894:	learn: 0.7616245	total: 47m 33s	remaining: 5m 34s
895:	learn: 0.7614679	total: 47m 36s	remaining: 5m 31s
896:	learn: 0.7614587	total: 47m 39s	remaining: 5m 28s
897:	learn: 0.7614211	total: 47m 42s	remaining: 5m 25s
898:	learn: 0.7613583	total: 47m 46s	remaining: 5m 21s
899:	learn: 0.7613287	total: 47m 49s	remaining: 5m 18s
900:	learn: 0.7613137	total: 47m 52s	remaining: 5m 15s
901:	learn: 0.7612676	total: 47m 55s	remaining: 5m 12s
902:	learn: 0.7611647	total: 47m 58s	remaining: 5m 9s
903:	learn: 0.7611323	total: 48m 1s	remaining: 5m 6s
904:	learn: 0.7610951	total: 48m 4s	remaining: 5m 2s
905:	learn: 0.7610628	total: 48m 8s	remaining: 4m 59s
906:	learn: 0.7610378	total: 48m 11s	remaining: 4m 56s
907:	learn: 0.7610046	total: 48m 14s	remaining: 4m 53s
908:	learn: 0.76

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(final_df['text'])

X_train, X_test, y_train, y_test = train_test_split(X_text, final_df['target_race'], test_size=0.2, random_state=42)


catboost_model = CatBoostClassifier()
catboost_model.fit(X_train, y_train)
Evaluate Model
y_pred = catboost_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



In [68]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

tfidf = TfidfVectorizer(max_features=39000) 
X_text = tfidf.fit_transform(final_df['text'])

X_train, X_test, y_train, y_test = train_test_split(X_text, final_df['target_religion'], test_size=0.2, random_state=42)

catboost_model = CatBoostClassifier(iterations=100, learning_rate=0.001, depth=8, task_type='CPU')  
catboost_model.fit(X_train, y_train)
y_pred = catboost_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



0:	learn: 3.4140329	total: 6.27s	remaining: 10m 20s
1:	learn: 3.3949505	total: 12.2s	remaining: 9m 59s
2:	learn: 3.3756711	total: 18.3s	remaining: 9m 53s
3:	learn: 3.3572316	total: 26.6s	remaining: 10m 37s
4:	learn: 3.3389625	total: 36.2s	remaining: 11m 26s
5:	learn: 3.3211413	total: 42.5s	remaining: 11m 6s
6:	learn: 3.3039957	total: 49.1s	remaining: 10m 52s
7:	learn: 3.2872389	total: 55.5s	remaining: 10m 37s
8:	learn: 3.2706197	total: 1m 1s	remaining: 10m 22s
9:	learn: 3.2543530	total: 1m 7s	remaining: 10m 11s
10:	learn: 3.2384535	total: 1m 14s	remaining: 10m
11:	learn: 3.2229087	total: 1m 21s	remaining: 9m 59s
12:	learn: 3.2075302	total: 1m 28s	remaining: 9m 53s
13:	learn: 3.1922285	total: 1m 34s	remaining: 9m 42s
14:	learn: 3.1775331	total: 1m 40s	remaining: 9m 30s
15:	learn: 3.1630974	total: 1m 46s	remaining: 9m 19s
16:	learn: 3.1489974	total: 1m 52s	remaining: 9m 9s
17:	learn: 3.1351430	total: 1m 59s	remaining: 9m 6s
18:	learn: 3.1215753	total: 2m 6s	remaining: 8m 57s
19:	learn: 3

In [60]:
import numpy as np

X_input_ids = X_text_tokenized.input_ids.numpy()
X_attention_mask = X_text_tokenized.attention_mask.numpy()
y = final_df['target_race'].values

X_train_input_ids, X_test_input_ids, X_train_attention_mask, X_test_attention_mask, y_train, y_test = train_test_split(
    X_input_ids, X_attention_mask, y, test_size=0.2, random_state=42
)
X_train = {'input_ids': torch.tensor(X_train_input_ids), 'attention_mask': torch.tensor(X_train_attention_mask)}
X_test = {'input_ids': torch.tensor(X_test_input_ids), 'attention_mask': torch.tensor(X_test_attention_mask)}


In [59]:
print("X_text_tokenized shape:", X_text_tokenized.input_ids.shape)
print("Target variable shape:", final_df['target_race'].shape)


X_text_tokenized shape: torch.Size([39565, 147])
Target variable shape: (39565,)
